In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import missingno as msno
from scipy import stats

In [ ]:
train = pd.read_csv('../input/water-potability/water_potability.csv')

In [ ]:
train

In [ ]:
train.info()

In [ ]:
train.describe().style.background_gradient(cmap = "Blues")

In [ ]:
train.duplicated().sum()

In [ ]:
msno.bar(train, figsize = (16,5),color = "#483D8B")
plt.show()

In [ ]:
for column_name in train.columns:
    unique_values = len(train[column_name].unique())
    print("Feature '{column_name}' has '{unique_values}' unique values".format(column_name = column_name,
                                                                                         unique_values=unique_values))

In [ ]:
train.columns

In [ ]:
numeric_features=['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity',
       'Organic_carbon', 'Trihalomethanes', 'Turbidity']

In [ ]:
import warnings
warnings.filterwarnings('ignore')
fig,ax = plt.subplots(len(numeric_features),4,figsize=(30,20))
for index,i in enumerate(numeric_features):
    sns.distplot(train[i],ax=ax[index,0],color='green')
    sns.boxplot(train[i],ax=ax[index,1],color='yellow')
    sns.violinplot(train[i],ax=ax[index,2],color='purple')
    stats.probplot(train[i],plot=ax[index,3])
    
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.suptitle("Visualizing continuous columns (train dataset)",fontsize=30)

In [ ]:
fig, ax1 = plt.subplots(figsize=(20,10))
graph = sns.countplot(ax=ax1,x = 'Potability' , data = train,palette='pastel')
graph.set_xticklabels(graph.get_xticklabels(),rotation=90)
for p in graph.patches:
    height = p.get_height()
    graph.text(p.get_x()+p.get_width()/2., height + 0.1,height ,ha="center")

In [ ]:
train.isnull().sum()

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
imputer = KNNImputer(n_neighbors=10, weights="uniform")

In [ ]:
l=imputer.fit_transform(train)

In [ ]:
train1=pd.DataFrame(l,columns=train.columns)

In [ ]:
train1

In [ ]:
train1.isnull().sum()

In [ ]:
# Shapiro-Wilk Test
from scipy.stats import shapiro
# normality test
for feature in numeric_features:
	stat, p = shapiro(train1[feature])
	print('Statistics=%.3f, p=%.3f' % (stat, p))
	# interpret
	alpha = 0.05
	if p > alpha:
		print(f'Sample looks Gaussian with {feature} (fail to reject H0)')
	else:
		print(f'Sample does not look Gaussian with {feature} (reject H0)')

In [ ]:
from scipy.stats import mannwhitneyu
for feature in numeric_features:
    stat, p = mannwhitneyu(train1['Potability'], train1[feature])
    print('Statistics=%.3f, p=%.3f' % (stat, p))
    alpha = 0.05
    if p > alpha:
		    print('There are no significant differences (fail to reject H0)')
    else:
		    print('The sample distributions are not equal (reject H0)')

In [ ]:
correlation = train1.corr()
print(correlation['Potability'].sort_values(ascending = False),'\n')

In [ ]:
k= 18
cols = correlation.nlargest(k,'Potability')['Potability'].index
print(cols)
cm = np.corrcoef(train1[cols].values.T)
mask = np.triu(np.ones_like(train1.corr()))
f , ax = plt.subplots(figsize = (14,12))
sns.heatmap(cm,mask=mask, vmax=.8, linewidths=0.01,square=True,annot=True,cmap='viridis',
            linecolor="white",xticklabels = cols.values ,annot_kws = {'size':12},yticklabels = cols.values)

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
clf = LocalOutlierFactor(n_neighbors=5, contamination='auto')
y_pred = clf.fit_predict(train1) 

In [ ]:
train1['Out']=y_pred

In [ ]:
Out=train[train1['Out']!=1]

In [ ]:
Out.shape

In [ ]:
train2=train1[train1['Out']==1]

In [ ]:
train2=train2.drop('Out',axis=1)

In [ ]:
from imblearn.over_sampling import SMOTE 

In [ ]:
oversample = SMOTE()
features, labels=  oversample.fit_resample(train2.drop(["Potability"],axis=1),train2["Potability"])

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
names = features.columns
d = scaler.fit_transform(features)

scaled_df = pd.DataFrame(d, columns=names)
scaled_df.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb

from sklearn.metrics import precision_score,recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(scaled_df,labels,test_size=0.33,random_state=42)

In [ ]:
models = [RandomForestClassifier(), KNeighborsClassifier(), SVC(), LogisticRegression(),xgb.XGBClassifier()]
scores = dict()

for m in models:
    m.fit(X_train, y_train)
    y_pred = m.predict(X_test)

    print(f'model: {str(m)}')
    print(classification_report(y_test,y_pred, zero_division=1))
    print('-'*30, '\n')

In [ ]:
!pip install optuna

In [ ]:
import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
def objective(trial: Trial,X,y) -> float:
    
    param = {
            'n_estimators' : trial.suggest_int("n_estimators",100,1500),
            'max_features' : trial.suggest_categorical("max_features",["auto","sqrt"]),
            'max_depth' : trial.suggest_int("max_depth",5,80,log = True),
            'min_samples_split' : trial.suggest_int("min_samples_split",2,15),
            'min_samples_leaf' : trial.suggest_int("min_samples_leaf",1,9),
            'bootstrap' : trial.suggest_categorical("bootstrap",[True,False])
            }
    
    model = RandomForestClassifier(**param)
    
    return cross_val_score(model, X, y, cv=5).mean()

In [ ]:
study = optuna.create_study(direction='maximize',sampler=TPESampler())
study.optimize(lambda trial : objective(trial,X_train,y_train),n_trials= 20)

In [ ]:
from optuna import visualization
print('Best trial: CV_score= {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))
hist = study.trials_dataframe()
hist.head()

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
Best_trial = study.best_trial.params

In [ ]:
best_clf2=RandomForestClassifier(**Best_trial)
best_clf2.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
cm = confusion_matrix(best_clf2.predict(X_test),y_test)
disp = ConfusionMatrixDisplay(cm, display_labels=["0","1"])
disp.plot()
plt.title("Confusion Matrix")
plt.show()

In [ ]:
feature_importances=best_clf2.feature_importances_
feature_importances_df=pd.DataFrame({'features':list(X_train), 'feature_importances':feature_importances})
feature_importances_df=pd.DataFrame({'features':list(X_train), 'feature_importances':feature_importances})
feature_importances_df.sort_values('feature_importances',ascending=False)